# Example


### Create a Relation

A relation R can be created using `pysdql.Relation(name='R')`.

In [1]:
import pysdql

r = pysdql.Relation(name='R')

print(r)

R


### Iteration

The iteration environment is automatically generated for relations in the construction method `__init__`, which can be accessed using `R.iter_expr`. By default, the iteration expression is presented as a key-value pair. 

In [2]:
r.iter_expr

sum (<r_k, r_v> in R)

### Select Columns

Columns of relation R can be selected using `R[['col1', 'col2', ..., 'coln']]`. This is implemented by overloading `__getitem__` method.

In [3]:
cols_a = r[['a']]

sum (<r_k, r_v> in R) { < a=r_k.a > }


In [4]:
cols_bc = r[['b', 'c']]

sum (<r_k, r_v> in R) { < b=r_k.b, c=r_k.c > }


### ColUnit

ColUnit is designed for creating conditional expressions by performing arithmetic operations ( +, -, *, / ) and logical operations ( &, | ). A ColUnit object can be created by `R['col']` or `R.col`, where `R` is a relation object and `col` is the column name. This is implemented by overloading `__getitem__` and `__getattr__` methods.

In [5]:
col_a = r['a']

col_a, type(col_a)

(r_k.a, pysdql.core.dtypes.ColumnUnit.ColUnit)

In [6]:
col_b = r.b

col_b, type(col_b)

(r_k.b, pysdql.core.dtypes.ColumnUnit.ColUnit)

### Conditional

Considering tpch-19, the SQL expression can be translated to the following conditial expression.

`p_partkey = l_partkey
and p_brand = ':1'
and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
and l_quantity >= :4 and l_quantity <= :4 + 10
and p_size between 1 and 5
and l_shipmode in ('AIR', 'AIR REG')
and l_shipinstruct = 'DELIVER IN PERSON'`

In [7]:
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)

c1 = (part['p_partkey'] == lineitem['l_partkey']) \
     & (part['p_brand'] == ':1') \
     & (part['p_container'].is_in(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (lineitem['l_quantity'] >= ':4') \
     & (lineitem['l_quantity'] <= ':4 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 5) \
     & (lineitem['l_shipmode'].is_in(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c1

p_k.p_partkey == l_k.l_partkey && p_k.p_brand == ":1" && (p_k.p_container == "SM CASE" || p_k.p_container == "SM BOX" || p_k.p_container == "SM PACK" || p_k.p_container == "SM PKG") && l_k.l_quantity >= ":4" && l_k.l_quantity <= ":4 + 10" && p_k.p_size > 1 && p_k.p_size < 5 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && l_k.l_shipinstruct == "DELIVER IN PERSON"

# TPCH - 1

In [8]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
lineitem = lineitem[lineitem['l_shipdate'] <= ':date']
lineitem = lineitem.aggr(l_returnflag=lineitem['l_returnflag'],
                         l_linestatus=lineitem['l_linestatus'],
                         sum_qty=(lineitem['l_quantity'], 'sum'),
                         sum_base_price=(lineitem['l_extendedprice'], 'sum'),
                         sum_disc_price=(lineitem['l_extendedprice'] * (1 - lineitem['l_discount']), 'sum'),
                         sum_charge=(lineitem['l_extendedprice'] * (1 - lineitem['l_discount']) * (1 + lineitem['l_tax']), 'sum'),
                         avg_qty=(lineitem['l_quantity'], 'avg'),
                         avg_price=(lineitem['l_extendedprice'], 'avg'),
                         avg_disc=(lineitem['l_discount'], 'avg'),
                         count_order=(lineitem['*'], 'count'))

let tmp = sum (<l_k, l_v> in lineitem) if (l_k.l_shipdate <= ":date") then { l_k } in
sum (<t_k, t_v> in tmp) { < l_returnflag=t_k.l_returnflag, l_linestatus=t_k.l_linestatus, sum_qty=t_k.l_quantity * t_v, sum_base_price=t_k.l_extendedprice * t_v, sum_disc_price=(t_k.l_extendedprice * (1 - t_k.l_discount)) * t_v, sum_charge=((t_k.l_extendedprice * (1 - t_k.l_discount)) * (1 + t_k.l_tax)) * t_v, count_order=t_v > }


# TPCH - 2

In [9]:
import pysdql

part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)
supplier = pysdql.Relation(name='supplier', cols=pysdql.SUPPLIER_COLS)
partsupp = pysdql.Relation(name='partsupp', cols=pysdql.PARTSUPP_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)
region = pysdql.Relation(name='region', cols=pysdql.REGION_COLS)

# TPCH - 3 ✅

In [10]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)

r = pysdql.merge(customer, orders, lineitem,
                 on=((customer['c_custkey'] == orders['o_custkey'])
                     & (orders['o_orderkey'] == lineitem['l_orderkey'])))

r = r[(customer['c_mktsegment'] == ':1') & (orders['o_orderdate'] < ':2') & (lineitem['o_orderdate'] > ':2')]

r = r.aggr(l_orderkey=lineitem['l_orderkey'],
           o_orderdate=orders['o_orderdate'],
           o_shippriority=orders['o_shippriority'])

let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) if(c_k.c_custkey == o_k.o_custkey && o_k.o_orderkey == l_k.l_orderkey)
  { concat(concat(c_k, o_k), l_k) -> c_v * o_v * l_v } in
let tmp = sum (<r_k, r_v> in R) if (c_k.c_mktsegment == ":1" && o_k.o_orderdate < ":2" && l_k.o_orderdate > ":2") then { r_k } in
sum (<t_k, t_v> in tmp) { < l_orderkey=t_k.l_orderkey, o_orderdate=t_k.o_orderdate, o_shippriority=t_k.o_shippriority > }


# TPCH - 4

# TPCH - 5 ✅

In [11]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
supplier = pysdql.Relation(name='supplier', cols=pysdql.SUPPLIER_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)
region = pysdql.Relation(name='region', cols=pysdql.REGION_COLS)

r = pysdql.merge(customer, orders, lineitem, supplier, nation, region,
                 on=(lineitem['l_orderkey'] == orders['o_orderkey'])
                    & (lineitem['l_suppkey'] == supplier['s_suppkey'])
                    & (customer['c_nationkey'] == supplier['s_nationkey'])
                    & (supplier['s_nationkey'] == nation['n_nationkey'])
                    & (nation['n_regionkey'] == region['r_regionkey']))

r = r[(region['r_name'] == ':1')
      & (orders['o_orderdate'] >= ':2')
      & (orders['o_orderdate'] >= ':2 + 1 year')]

r = r.aggr(n_name=nation['n_name'],
           revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum')
           )


let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) sum (<s_k, s_v> in supplier) sum (<n_k, n_v> in nation) sum (<r_k, r_v> in region) if(l_k.l_orderkey == o_k.o_orderkey && l_k.l_suppkey == s_k.s_suppkey && c_k.c_nationkey == s_k.s_nationkey && s_k.s_nationkey == n_k.n_nationkey && n_k.n_regionkey == r_k.r_regionkey)
  { concat(concat(concat(concat(concat(c_k, o_k), l_k), s_k), n_k), r_k) -> c_v * o_v * l_v * s_v * n_v * r_v } in
let tmp = sum (<r_k, r_v> in R) if (r_k.r_name == ":1" && o_k.o_orderdate >= ":2" && o_k.o_orderdate >= ":2 + 1 year") then { r_k } in
sum (<t_k, t_v> in tmp) { < n_name=t_k.n_name, revenue=(l_k.l_extendedprice * (1 - l_k.l_discount)) * t_v > }


# TPCH - 6 ✅

In [12]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)

lineitem = lineitem[(lineitem['l_shipdate'] >= ':1')
                    & (lineitem['l_shipdate'] < ':1 + 1 year')
                    & (lineitem['l_discount'] > ':2 - 0.01')
                    & (lineitem['l_discount'] < ':2 + 0.01')
                    & (lineitem['l_quantity'] > ':3')]

lineitem = lineitem.aggr(revenue=(lineitem['l_extendedprice'] * lineitem['l_discount'], 'sum'))

let tmp = sum (<l_k, l_v> in lineitem) if (l_k.l_shipdate >= ":1" && l_k.l_shipdate < ":1 + 1 year" && l_k.l_discount > ":2 - 0.01" && l_k.l_discount < ":2 + 0.01" && l_k.l_quantity > ":3") then { l_k } in
sum (<t_k, t_v> in tmp) { < revenue=(t_k.l_extendedprice * t_k.l_discount) * t_v > }


TPCH - 7

TPCH - 8

TPCH - 9

# TPCH - 10 ✅

In [13]:
import pysdql

customer = pysdql.Relation(name='customer', cols=pysdql.CUSTOMER_COLS)
orders = pysdql.Relation(name='orders', cols=pysdql.ORDERS_COLS)
lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
nation = pysdql.Relation(name='nation', cols=pysdql.NATION_COLS)

r = pysdql.merge(customer, orders, lineitem, nation,
                 on=(customer['c_custkey'] == orders['o_custkey'])
                    & (lineitem['l_orderkey'] == orders['o_custkey'])
                    & (customer['c_nationkey'] == nation['n_nationkey'])
                 )

r = r[(orders['o_orderdate'] >= ':1')
      & (orders['o_orderdate'] >= ':1 + 3 month')
      & (lineitem['l_returnflag'] == 'R')]

r = r.aggr(c_custkey=customer['c_custkey'],
           c_name=customer['c_name'],
           c_acctbal=customer['c_acctbal'],
           c_address=customer['c_address'],
           c_phone=customer['c_phone'],
           c_comment=customer['c_comment'],
           revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

let R = sum (<c_k, c_v> in customer) sum (<o_k, o_v> in orders) sum (<l_k, l_v> in lineitem) sum (<n_k, n_v> in nation) if(c_k.c_custkey == o_k.o_custkey && l_k.l_orderkey == o_k.o_custkey && c_k.c_nationkey == n_k.n_nationkey)
  { concat(concat(concat(c_k, o_k), l_k), n_k) -> c_v * o_v * l_v * n_v } in
let tmp = sum (<r_k, r_v> in R) if (o_k.o_orderdate >= ":1" && o_k.o_orderdate >= ":1 + 3 month" && l_k.l_returnflag == "R") then { r_k } in
sum (<t_k, t_v> in tmp) { < c_custkey=t_k.c_custkey, c_name=t_k.c_name, c_acctbal=t_k.c_acctbal, c_address=t_k.c_address, c_phone=t_k.c_phone, c_comment=t_k.c_comment, revenue=(l_k.l_extendedprice * (1 - l_k.l_discount)) * t_v > }


TPCH - 11

TPCH - 12

TPCH - 13

TPCH - 14

TPCH - 15

TPCH - 16

TPCH - 17

TPCH - 18

# TPCH - 19 ✅

In [14]:
import pysdql

lineitem = pysdql.Relation(name='lineitem', cols=pysdql.LINEITEM_COLS)
part = pysdql.Relation(name='part', cols=pysdql.PART_COLS)

c1 = (part['p_partkey'] == lineitem['l_partkey']) \
     & (part['p_brand'] == ':1') \
     & (part['p_container'].is_in(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (lineitem['l_quantity'] >= ':4') \
     & (lineitem['l_quantity'] <= ':4 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 5) \
     & (lineitem['l_shipmode'].is_in(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c2 = (part['p_partkey'] == lineitem['l_partkey']) \
     & (part['p_brand'] == ':2') \
     & (part['p_container'].is_in(('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK'))) \
     & (lineitem['l_quantity'] >= ':5') \
     & (lineitem['l_quantity'] <= ':5 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 10) \
     & (lineitem['l_shipmode'].is_in(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c3 = (part['p_partkey'] == lineitem['l_partkey']) \
     & (part['p_brand'] == ':3') \
     & (part['p_container'].is_in(('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG'))) \
     & (lineitem['l_quantity'] >= ':6') \
     & (lineitem['l_quantity'] <= ':6 + 10') \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 15) \
     & (lineitem['l_shipmode'].is_in(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

r = pysdql.merge(lineitem, part, on=c1 | c2 | c3)

r = r.aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

let R = sum (<l_k, l_v> in lineitem) sum (<p_k, p_v> in part) if(((p_k.p_partkey == l_k.l_partkey && p_k.p_brand == ":1" && (p_k.p_container == "SM CASE" || p_k.p_container == "SM BOX" || p_k.p_container == "SM PACK" || p_k.p_container == "SM PKG") && l_k.l_quantity >= ":4" && l_k.l_quantity <= ":4 + 10" && p_k.p_size > 1 && p_k.p_size < 5 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && l_k.l_shipinstruct == "DELIVER IN PERSON") || (p_k.p_partkey == l_k.l_partkey && p_k.p_brand == ":2" && (p_k.p_container == "MED BAG" || p_k.p_container == "MED BOX" || p_k.p_container == "MED PKG" || p_k.p_container == "MED PACK") && l_k.l_quantity >= ":5" && l_k.l_quantity <= ":5 + 10" && p_k.p_size > 1 && p_k.p_size < 10 && (l_k.l_shipmode == "AIR" || l_k.l_shipmode == "AIR REG") && l_k.l_shipinstruct == "DELIVER IN PERSON")) || (p_k.p_partkey == l_k.l_partkey && p_k.p_brand == ":3" && (p_k.p_container == "LG CASE" || p_k.p_container == "LG BOX" || p_k.p_container == "LG PACK" || p_k.p

TPCH - 20

TPCH - 21

TPCH - 22